# Convolution

이미지 분야의 딥러닝에서 가장 많이 쓰이는 기법으로 CNN이 있다. 그 중 Convolution에 대해서 알아보자



## 1. 신호처리에서의 convolution

우선 신호처리 분야에서의 Convolution은 다음과 같다. 

<img src="./pic/1.PNG" width=80%/>

개략적으로 설명하면 위 그림을 보면 처음에 f와 g의 두 신호가 존재한다. 위 신호중에서 f나 g중 하나의 신호를 축을 기준으로 뒤집는다.

위 그림에서는 g가 뒤집혔다. 이후에 뒤집은 신호를 쭉 슬라이딩 하면서 f와 겹치는 면적의 값을 구한다. 

아래 그림을 참고하면 아래에서 삼각형 모양이 convolution을 한 결과이다.<bf><bf>


<img src="./pic/2.GIF" width=80%/>

## 2. 딥러닝에서의 convolution

사실 위의 내용을 깊게 알필요는 없으니 다시 딥러닝으로 돌아오면 딥러닝에서의 Convolution은 위와 조금 다르다.

우선 딥러닝에서는 g의 역할이 filter(kernel) 일텐데 실제 구현시 이를 뒤집지 않고 구현하게 된다. 

(** 엄밀하게 말하면 딥러닝에서의 convolution은 cross-correlation 이다.)

또한 이미지는 Depth가 존재하므로 3차원의 공간에서 진행된다. 이를 아래 애니메이션으로 이해해보자.

<img src="./pic/3.GIF" width=80%/>

위 애니메이션을 보면 5 X 5 X 3 의 이미지가 있다. 이 이미지를 4개의 3 X 3 X 3 의 필터를 이용해서 convolution 함을 알 수 있다. 

오른쪽 그림은 최종적으로 나오는 Activation map으로 3 X 3 X 4 의 크기를 갖게 된다.

즉, R G B 각각의 이미지에서 필터는 각각 9개의 곱한 값을 갖게 되어 총 27개의 곱셈 값의 합을 통해 하나의 값을 얻게 된다.

그러면 convolution은 왜 하는걸까?

우선 convolution을 보면 하나의 filter의 값이 고정된 채로 슬라이딩을 하면서 진행이 되는데 이 말은 픽셀이 filter의 가중치를 공유한다는 말이다.

이렇게 하게 되면 translation invariance(위치에 대한 불변성)이 생기고, 네트워크에서 parameter의 양이 줄어들게 되고, 사진 자체를 데이터로 이용하므로 공간적인 인식도 가능하게 해준다. 

# Problem 1


이제 실제 예시를 들어보겠다. 다만, 3차원일 경우 너무 복잡해 지는 것을 막기 위해서 Depth가 1인 2차원 평면의 예제로 보면 다음과 같다. 

<img src="./pic/4.GIF" width=80%/>

여기서 처음 값을 계산하는데 다음과 같이 계산한다.

```
(1 * -1) + (2 * 1) + (0 * 2) + (1 * 1) + (0 * 1) + (0 * 0) + (2 * -1)+ (2 * 0) = -2
```

이를 실제로 구현해보자.

In [2]:
import numpy as np

x = np.array([[1, 2, 0, 3, 1],
              [1, 0, 0, 2, 2],
              [2, 1, 2, 1, 1],
              [0, 0, 1, 0, 0],
              [1, 2, 1, 1, 1]]).reshape(1,5,5,1).astype(np.float32)

filters = np.array([[-1,  1, 2],
                    [ 1,  1, 0],
                    [-1, -2, 0]]).reshape(3,3,1,1).astype(np.float32)

우선 x는 입력 데이터이다. 하지만 reshape를 통해 `[1, 5, 5, 1]` 로 만들었는데 의미는 다음과 같다.
  * the batch (1) 
  * the height (5) 
  * the width (5) 
  * the number of channels (1) 
  

마찬가지로 filters도 `[3, 3, 3, 1]` 인데 위 숫자도 다음과 같다. 

  * the height of the filter (3) 
  * the width of the filter (3) 
  *  the number of channels in the _input_ (1)
  *  the number of channels in the output (1)
  
여기서 마지막 숫자인 output channels 는 filter의 개수와도 같다.


### 텐서플로의 conv2d를 이용해서 결과를 구해보자.

In [3]:
import tensorflow as tf

In [13]:
with tf.Session() as sess:
    out = tf.nn.conv2d(input = x,
                       filter = filters,
                       strides = [1, 1, 1, 1],
                       padding = 'VALID')
    result = sess.run(out)

    print(result.squeeze())


[[-2. -1.  3.]
 [ 2.  5.  8.]
 [-2.  0. -1.]]
